# TXINE. Práctica3: Extracción de datos dunha rede social
### Adolfo Núñez Fernández, Novembro 2015 
## Requirimentos

Extracción de datos dunha rede social para a súa posterior análise. Traballaremos con Reddit para construír unha colección de datos, que conteña envíos (posts e/ou comentarios) emitidos por usuarios en Reddit. Tentarase extraer o máximo número de posts e comentarios dunha das subcomunidades de Reddit (subreddits). Por exemplo "science", "politics", ... 
O programa, empregando as posibilidades da API de Reddit, definirá alomenos dous modos de extraer contidos: 

* os últimos contidos (extraendo o máximo número que permite Reddit) [NEW] 
* os contenidos máis populares (acorde ás valoraciones de Reddit) [TOP]
* os contidos máis candentes [HOT]

Este conxunto de entradas xunto cos seus comentarios asociados serán almacenados en disco en un formato XML. Este esquema XML permitirá almacenar toda a información dispoñible (alomenos, título, contido, data, tipo de entrada -post ou comentario-) nun único arquivo

Finalmente realizarase un simple procesamento do corpus xerado no ficheiro XML para vectorizar a colección e amosar os termos con maior ponderación tf/idf. Para iso, filtrando stopwords e todas aquelas palabras que aparezan en menos de 10 documentos para vectorizar la colección e amosar os 10 termos máis "centrais" na colección, entendendo como máis centrais aqueles que teñan a maior suma acumulada de tf/idf sobre todos os documentos.

## Explicación do código

O programa estructúrase a partires dunha serie de menús e información requirida ao usuario. A función **main** ofrece un menú principal de opcións ao usuario, onde poderá escoller entre descargar e almacenar unha nova colección ou ler e analizar un arquivo XML almacenado previamente. No caso de escoller unha nova adquisición de datos, un novo menú permite escoller entre as tres opciósn posibles especificadas nos requisitos: **TOP/HOT/NEW**. Ademáis o usuario pode especificar o límite de documentos desexados (con opción de adquirir o máximo que permita **Reddit**) e asignar un nome ao ficheiro XML onde se gardará a colección. O programa dispón tamén dunha opción no código (comentada por defecto pola súa demora no tempo de execución), que permite a adquisición reiterada de comentarios sobre un post, e non só os que trae Reddit por defecto. 

Na segunda opción, o usuario especificará un ficheiro a ler e analizar. O programa automaticamente filtrará aquelas palabras  que aparezan en menos de 10 documentos e eliminará unha lista de stopwords, que son a unión dunha colección por defecto máis unha colección seleccionada tras a inspección experta de varias execucións sobre distintos subreddits. O programa amosará os 10 termos centrais da colección, tal como se soliciata nos requirimentos. O código está estruturado en funciósn independentes por funcionalidade, que se describen a continuación. 

### Imports



In [ ]:
import sys
import praw
import xml.etree.ElementTree as ET 
from datetime import datetime  
from sklearn.feature_extraction.text import TfidfVectorizer

As librarías empregadas son:

* **sys:** Para acceder ás excepcións do sistema.

* **praw:** Para acceder á API de Reddit.

* **xml.etree.ElementTree:** Permite manexar obxectos estructurados en forma de árbore. En concreto emprégase para a manipulación, escritura e lectura do ficheiro XML.

* **datime:** Libraría estandar de python empregada para medir tempso de execución do programa.

* **TfidfVectorizer:** Obxecto da libraria **sklearn** que permite a vectorización e análise de corpus de texto.

### Main e funcións menu(), mainMenu()

A función **main** permite a interacción do usuario co programa a través dun menú de tres opcións:
* [1] Obtención de datos e gardado a XML
* [2] Lectura de XML e análise Tfidf
* [3] Saída do programa.

O programa presenta o menú principal (**mainMenu()**) mentres non se escolle algunha das opcións válidas. 

Na opción 1, o programa conéctase á API de reddit mediante previa identificación a través dun **user_agent**. Establecida a conexión, o usuario debe escoller o límite de post a solicitar á API. Posteriormente preséntaselle un submenu (**menu()**) onde o usuario escollerá o tipo de posts que desexa:
* [1] Hot: Máis candentes
* [2] Top: Máis valorados
* [3] New: Máis novos
* [4] Cancelar: volta ao menú principal

Finalmente deberá introducir o nome do ficheiro XML onde vai gardar a información. Se o usuario non proporciona ningún, automáticamente créase un por defectoco nome xerado polas variables: <TIPOPOST>_<nomeSubreddit>_<limite>. O programa continua coa chamada á función **data2XML()** que se encarga da recollida e almacenamento de datos.

Na segunda opción, solicítaselle ao usuario o ficheiro a analizar, para proceder á súa lectura, análise e visualización de resultados, coas funcións **getCorpusFromXML()**, **vectorizaCorpus()** e **showResults** respectivamente. É aquí onde están definidas tamén as opcións de filtrado coma o número mínimo de documentos onde deben aparecer as palabras do corpus e o número de resultados a amosar tras a análise. 

A opción 3 remata a execución do programa, presentando unha mensaxe de despedida por pantalla. 

In [ ]:
def mainMenu():
    print("\nEscolla unha opción:\n\
    [1] Obtención de datos e gardado a XML\n\
    [2] Lectura de XML e análise Tfidf\n\
    [3] Saír")
    
def menu():
    print("\nEscolla unha opción:\n\
    [1] Hot\n\
    [2] Top\n\
    [3] New\n\
    [4] Cancelar")

if __name__ == "__main__":
    ''' Programa principal. Chamada ao menú de opcións '''
    
    mainOption = 0
    while mainOption != 3:
        
        if mainOption == 1:
            
            # Opción inicial para que entre no bucle
            opcion = 0
              
            while opcion != 4:
                if opcion in (1,2,3):
                    user_agent = "windows:com.exemplo.socialnetworkdataextraction:v1.0 (by /u/rebuldeiro)"
                    r = praw.Reddit(user_agent=user_agent)
                    #r.config.store_json_result = True
                    
                    limite = int(input('Límite de peticións (0 para maximo posible):'))
                    numPetic = str(limite)                    
                    if limite  == 0:
                        limite = None
                        numPetic = 'Max'
                    try:
                        subreddit = str(input('Comunidade (subreddit) a analizar: '))
                        comunidade = r.get_subreddit(subreddit, fetch=True)
                    except praw.errors.InvalidSubreddit:
                        print('Non existe a comunidade {0}\n'.format(subreddit))
                        sys.exit()
                        
                if opcion == 1: # HOT
                    print("Obtendo {0} posts {1} de {2}".format(numPetic, 'HOT', comunidade))
                    submissions = comunidade.get_hot(limit=limite) 
                    tipo = 'HOT'
                    print('[FEITO]')
                elif opcion == 2: # TOP
                    print("Obtendo {0} posts {1} de {2}".format(numPetic, 'TOP', comunidade))
                    submissions = comunidade.get_top(limit=limite)
                    tipo = 'TOP'                    
                    print('[FEITO]')
                elif opcion == 3: # NEW
                    print("Obtendo {0} posts {1} de {2}".format(numPetic, 'NEW', comunidade))
                    submissions = comunidade.get_new(limit=limite)
                    tipo = 'NEW'
                    print('[FEITO]')
                else:
                    pass
                # se a opción é unha da que nos interesa
                if opcion in (1,2,3):
                    fname = input('Nome de ficheiro XML a gardar (sen extensión): ')
                    if fname:
                        fname += '.xml'
                    else:
                        fname = '{0}_{1}_{2}.xml'.format(tipo, subreddit, limite)
                    
                    data2XML(submissions, fname)
                    break
                    
                #chamada ao menú
                menu()
                # Recóllese a opción do usuario
                opcion = int(input('Opción: '))
            
            mainOption = 0
        
        elif mainOption == 2:
            try:
                fxml = input('Nome de ficheiro (sen extensión): ') 
                corpus = getCorpusFromXML(fxml+'.xml')
                minDf = 10
                numResults = 10                
                vectorizer, invVoc, sumaTfidf = vectorizaCorpus(corpus, minDf)
                showResults(vectorizer, invVoc, sumaTfidf, numResults)
            except IOError as e:
                print("I/O error({0}): {1}".format(e.errno,'Non se atopou o ficheiro'))
            except:
                print ("Produciuse un erro inesperado:", sys.exc_info()[0])
                raise

            mainOption = 0
        else:
            pass
    
        #chamada ao menú principal
            mainMenu()
            # Recóllese a opción do usuario
            mainOption = int(input('Opción principal: '))        
    if mainOption == 3:
        print("Adeus")


### Función data2XML(posts, nomeFicheiro)

**DEF:** Función que convirte as submissions (posts) obtidas a un ficheiro XML. 

**Parámetros de entarda:** 
* **posts:** é un iterable de obxectos tipo **submission**, que é a colección de submissions (posts e comentarios) obtido mediante a API de Reddit (https://praw.readthedocs.org/en/stable/)
* **nomeFicheiro:** nome (ruta) do ficheiro co que se creará o XML onde se gardará a información.

**Funcionamento:**

Co módulo xml.etree.ElementTree (https://docs.python.org/3/library/xml.etree.elementtree.html)temos unha API para crear e parsear datos XML. A medida que percorremos as submissions imos creando elementos XML (etiquetas) engadíndoos á estructura. O que nos interesa para a nosa análise é unha estructura de textos. Crearemos unha estructura simple como a que se amosa a continuación, gardando a información relevante de cada post e cada comentario coma atributos da etiqueta XML correspondente:

In [ ]:
<Subreddit>
    <Post atributosPost>TextoPost
        <Comments>
            <Comment atributosComment>Texto comment</comment>
            <Comment atributosComment>Texto comment</comment>
            <Comment atributosComment>Texto comment</comment>
            ...
            <Comment atributosComment>Texto comment</comment>
        </Comments>
    </Post>
    <Post atributosPost>TextoPost
    ...
    </Post>
</Subreddit>            

Empezaremos polo elemento raíz

In [ ]:
rootNode = ET.Element('Subreddit')

De cada obxecto submission podemos extraer os seus comentarios e comentarios de comentarios con:

In [ ]:
comments = praw.helpers.flatten_tree(submission.comments)

A estratexia que seguiremos será almacenar a información do obxecto submission (post) e comment en diccionario de atributos que lle pasaremos ás etiquetas XML que imos construíndo. O único filtrado qeu faremos será de eliminar aqueles comentarios que non teñen autor ou non teñen contido. Ao rematar con todas as submissions pechamos a conexión a Reddit e gardamos o ficheiro a XML gracias de novo ás utilidades do módulo ElementTree. O código da función comentado é:

In [ ]:
def data2XML(submissions, fname):
    ''' Convirte as submissions obtidas a un ficheiro XML '''

    startTime = datetime.now()    
    rootNode = ET.Element('Subreddit')
    rootNode.append(ET.Comment('Xerado para TXINE. Novembro 2015'))
    
    
    i = 0
    
    for submission in submissions: # Cada submission é un post
        try:
            i += 1
            print("Engadindo post {0} ao XML\r".format(i),end='')        
            
            '''
            Descomentar só para obter máis comentarios (se existen) sobre o 
            post. Tarda máis: 10 posts ~ 13'30''  
            '''  
            #submission.replace_more_comments(limit=None, threshold=0)
            
            comments = praw.helpers.flatten_tree(submission.comments)
            
            # Gardamos os atributos que nos interesan de cada submission    
            atrSub = {'author':submission.author.name,
                      'date':str(submission.created),                  
                      'date_utc':str(submission.created_utc),
                      'id':str(submission.id),
                      'num_total_comments':str(submission.num_comments), #inclue os eliminados
                      'num_true_comments':str(len(comments)), #so comments e replys existentes
                      'title':submission.title,
                      'type':'post'
                      }
            elementoPost = ET.SubElement(rootNode, 'Post', atrSub)    
            elementoPost.text = str(submission.title)+' : '+str(submission.selftext) # concatenase titulo e texto
                             
            if len(comments) > 0: #se hai comentarios
                elementoComments = ET.SubElement(elementoPost,'Comments')
                for comment in comments: # Percorremos os comentarios (non aparecen os eliminados) 
                    if hasattr(comment,'body') and hasattr(comment,'author') and str(comment.author) != 'None':
                            atrCom = {'author': comment.author.name,
                                      'date':str(comment.created),                  
                                      'date_utc':str(comment.created_utc),
                                      'id':str(comment.id),
                                      'parent_id':str(comment.parent_id),
                                      'type':'comment' if comment.is_root else 'reply'
                                                }
                            elementoComment = ET.SubElement(elementoComments,'Comment',atrCom)
                            elementoComment.text = str(comment.body) 
        except Exception as e:
            print('Exception: {0}'.format(e))    
    r.http.close()
    # Gardado a ficheiro
    with open(fname, 'wb') as f:
        ET.ElementTree(rootNode).write(f, method='xml')

    print('\nTempo de extracción e almacenamento: {0}'.format(datetime.now() - startTime))

### Función getCorpusFromXML(nomeFicheiro)

**DEF:** Función encargada da obtención de información dun corpus textual a partires dun ficheiro XML.

**Parámetros de entrada:** 
* **nomeFicheiro:** nome (ou path) do ficheiro XML

**Parámetros de saída:**
* **corpus** lista de textos

**Funcionamento:**

As funcionalidades de **elementTree** facilítannos a tarefa de recuperación dos textos dun ficheiro XML e almacenamento nunha lista. A función *parsea* o ficheiro a unha estructura de tipo árbore e a partires do seu nodo raíz par aposteriormente obter todos os textos correspondentes aos **posts** e aos **comentarios** en sendas listas, que unidas conformarán a lista **corpus** a devolver.

In [ ]:
def getCorpusFromXML(oXML):
    # Lectura do ficheiro xml
    tree = ET.parse(oXML)
    root = tree.getroot()
    
    # Creamos un corpus para analizar con posts e comentarios 
    posts = [x.text for x in root.iter("Post")]
    comentarios = [x.text for x in root.iter("Comment")]
    
    corpus = posts + comentarios
    print('\nCorpus creado con {0} posts e {1} comentarios'.format(len(posts), len(comentarios)))
    return corpus

### Funcion vectorizaCorpus(corpus, minimoDocumentosAfiltrar)

**DEF:** Función que se encarga da vectorización do corpus de texto para a súa posterior análise. 

**Parámetros de entrada:**
* **corpus:** lista de documentos en formato texto
* **minimoDocumentosAfiltrar:** enteiro que indica o número mínimo de documentos nos que debe aparecer unha palabra para que sexa considerada relevante para análise. 

**Parámetros de saída:**
* **vectorizer:** vectorizador do corpus
* **invVoc:** diccionario de índices-termos para a presentación de resultados 
* **sumaTfidfun:** lista onde se almacenan as suma acumulada do tf/idf en tódolos documentos, empregada coma medida de centralización. 

**Comportamento:**

O primeiro que fai esta función é chamar a **TfidfVectorizer**, unha clase da libraría **scikit** (http://scikit-learn.org/stable/index.html) que serve para convertir unha colección de documentos (a nosa lista corpus) nunha matriz de características TF-IDF. Esta clase admite unha colección de *stopwords* para o idioma inglés, pero engadíronselle ademais unha serie de palabras recoñecidas coma irrelevantes mediante a observación dos resultados. Esta observación foi realizada sobre distintos subreddits, pois hai palabras de uso común que se repiten en moitos deles polo simple uso da linguaxe natural, e que non teñen que ver con palabras relevantes do tema a tratar. Algún exemplo poden ser palabras coma *thanks*, *yes*, *yeah*, *reddit*, *know*...

Unha vez engadidas as stopwords, calculáse a matriz de documentos-termos a partires do corpus, e créase un vocabulario invertido para indexar os termos, de cara á súa presentación. Tamén se calcula a suma acumulada en tódolos documentos do tf/idf para a ordenación de resultados.

In [ ]:
def vectorizaCorpus(corpus, minDf):
    ''' Vectoriza o corpus introducido filtrando as palabras que aparecen en 
    menos de minDf documentos'''
    try:
        vectorizer = TfidfVectorizer(min_df = minDf, lowercase=True, stop_words='english')
        
        # Definimos unha lista propoia de stopwords
        myStopwords = ['did','didn','does','doesn','don','just','isn', \
        'reddit', 'wasn','www','yeah','yes','like','able','thanks', \
        'know', 'think','ve', 'want','com','https','http',\
        'good', 'really', 'make', 'say', 'going', 'said', 'people','way', \
        'use']
        
        # engadimos as stop_words que queremos ao conxunto xa existente
        vectorizer.stop_words = vectorizer.get_stop_words().union(myStopwords)
        
        # calculamos a matriz de documentos-términos
        docTerms = vectorizer.fit_transform(corpus)
        
        # invertimos o vocabulario creando un diccionario de índices - termos
        invVoc = {v: k for k, v in vectorizer.vocabulary_.items()} 
        
        # buscamos os termos centrais, que son os que a suma acumulada de tf/idf en todos os documentos é maior
        sumaTfidf = docTerms.sum(axis=0).tolist()[0] #calculamos a suma por columnas da matriz de documentos-termos
    
        return  vectorizer, invVoc, sumaTfidf
    except Exception as e:
        print('\nOcorreu un problema: {0}'.format(e))
        sys.exit()

### Funcion showResults(vectorizer, invVoc, sumaTfidf, numResults)

**DEF:** Función que presenta os resultados ao usuario en forma de táboa por pantalla, segundo os requerimentos da práctica, é dicir, os 10 termos centrais, entendendo como máis centrais aqueles que teñan a maior suma acumulada de tf/idf sobre todos os documentos.

**Parámetros de entrada:**
* **vectorizer:** que posúe os valores idf para o corpus
* **invVoc:** diccionario que relaciona índices e termos, para poder relacionar a orde na lista coas palabras do corpus
* **sumaTfidf:** suma acumulada de tf/idf. Suma por columnas da matriz documentos-termos
* **numResults:** número de resultados a amosar ao usuario. Está fixado a 10 nesta práctica.

**Parámetros de saída:**
* **Non hai**

**Comportamento:**

Cos parámetros de entrada crea unha lista e mediante a combinación das funcións **sorted** e **getKey** (esta última definida para indicar a columna pola que se ordeará a lista), conv´´irtea nunha lista ordenada en orde descendente. Finalmente amosa tantos resultados como vñan indicados polo parámetro *numResults*

In [ ]:
def showResults(vectorizer, invVoc, sumaTfidf, numResults):
    # Almacenamos todo nunha lista para poder amosar os resultados ordeados
    listaResultados = []
    for i in range(len(sumaTfidf)):
        listaResultados.append([invVoc[i], sumaTfidf[i], vectorizer.idf_[i]])
    
    def getKey(item):
        ''' Función que devolve o numero de columna polo que imos ordear a lista '''
        return item[1]
    
    listaResultadosOrdenada = sorted(listaResultados, key = getKey, reverse = True)
    print('{:15s} {:>10s} {:>10s}\n{:45s}'.format('Palabra','Suma','Idf','-'*45))
    for i in range(numResults):
        print('{:15s} {:10.2f} {:10.2f}'.format(listaResultadosOrdenada[i][0],
                                     listaResultadosOrdenada[i][1],listaResultadosOrdenada[i][2]))

###  Exemplo de resultados:

Amósanse a continuación catro saídas de resultados, con tantos resultados como foi posible obter. Dúas para ficheiros do tipo HOT e dúas para ficheiros de tipo TOP ambas sobre dous subreddits distintos: *politics* e *science*. 

Obsérvase que quizáis se podería ter metido coma *stopword* algun termo coma *time*, *right*, ... pero pode que nestes contextos sexa un termo relevante.

In [ ]:
Nome de ficheiro (sen extensión): TOP_science_None

Corpus creado con 55 posts e 830 comentarios
Palabra               Suma        Idf
---------------------------------------------
blood                41.05       3.28
sex                  26.76       3.71
time                 22.75       3.51
depression           22.32       3.68
pigeons              21.90       4.03
study                18.82       4.07
donate               17.74       4.12
week                 15.81       4.03
years                15.73       3.87
inflammation         15.19       4.15

----------------------------------------------------------------------------------------------------------------------------
Nome de ficheiro (sen extensión): HOT_science_None

Corpus creado con 673 posts e 11272 comentarios
Palabra               Suma        Idf
---------------------------------------------
time                145.38       3.48
study               127.44       3.75
water               117.79       4.28
years               112.32       3.83
science             110.05       4.08
work                 98.28       3.93
new                  94.31       4.06
actually             94.12       3.95
research             92.99       4.05
article              87.83       4.19

----------------------------------------------------------------------------------------------------------------------------
Nome de ficheiro (sen extensión): HOT_politics_None

Corpus creado con 954 posts e 23668 comentarios
Palabra               Suma        Idf
---------------------------------------------
trump               275.54       4.11
right               259.29       3.76
refugees            258.81       3.98
sanders             209.94       4.19
bernie              201.36       4.33
time                191.01       4.06
obama               189.39       4.33
money               188.49       4.18
vote                182.89       4.45
clinton             182.35       4.37

----------------------------------------------------------------------------------------------------------------------------
Nome de ficheiro (sen extensión): TOP_politics_None

Corpus creado con 293 posts e 4978 comentarios
Palabra               Suma        Idf
---------------------------------------------
trump               105.12       3.64
right                75.39       3.68
clinton              67.20       4.04
bernie               63.27       4.12
hillary              62.22       4.07
sanders              59.54       4.06
vote                 54.80       4.25
obama                51.95       4.30
government           47.98       4.18
republican           46.13       4.36